In [2]:
import os
from sqlalchemy import create_engine
import pybaseball as pyb
import pybaseball.cache # Ensure caching is imported
import pandas as pd
from dotenv import load_dotenv
import time
from datetime import date, timedelta
from sqlalchemy.engine import Engine
from sqlalchemy import text
import datetime
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, TimeoutException
from rapidfuzz import process
import re
import numpy as np
from io import StringIO
import pylahman
import statsapi

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

Get probable starter of the day

In [ ]:
# import statsapi
# import pandas as pd

# 1. Get today's date automatically
today_str = datetime.now().strftime('%m/%d/%Y')

def get_daily_starters(date_str):
    # 1. Fetch all games for the specified date
    # No team ID means it pulls all 30 teams
    schedule = statsapi.schedule(date=  date_str)
    
    games_list = []
    
    for game in schedule:
        # Extract the core info for the Matchup Predictor
        game_data = {
            "game_id": game.get("game_id"),
            "home_team": game.get("home_name"),
            "away_team": game.get("away_name"),
            "home_pitcher": game.get("home_probable_pitcher", "TBD"),
            "away_pitcher": game.get("away_probable_pitcher", "TBD"),
            "status": game.get("status"),
            "venue": game.get("venue_name")
        }
        games_list.append(game_data)
    
    # 2. Convert to DataFrame
    df = pd.DataFrame(games_list)
    return df

# Test with June 15, 2025
df_starters = get_daily_starters('6/15/2025')
# TODO Need to create a function if the table is empty
#df_starters = get_daily_starters(today_str)

# Display the first few rows
print(df_starters.head())

# Integrating with your "Matchup Predictor"
# Now that you have this DataFrame, you can loop through the home_pitcher and away_pitcher columns.

# For each name:

# Look up their Archetype in your database.

# Highlight the "Best Matchups" for that day 
# (e.g., "Today, 3 teams are facing 'Soft Tossers'—check your O's hitters' 
# luck scores against that archetype").

Empty DataFrame
Columns: []
Index: []


Get today's lineup

In [ ]:
import statsapi
import pandas as pd

# 1. Get today's date automatically
today_str = datetime.now().strftime('%m/%d/%Y')

def get_lineups_to_df(date_str):
    # 1. Get all games for the day to find game_ids
    schedule = statsapi.schedule(date=date_str)
    
    lineup_records = []

    for game in schedule:
        game_id = game['game_id']
        team_names = {
            'home': game['home_name'],
            'away': game['away_name']
        }
        
        # 2. Pull boxscore data which contains the lineup
        try:
            box = statsapi.boxscore_data(game_id)
            
            for side in ['home', 'away']:
                # The 'battingOrder' list contains player IDs in order (1-9)
                order_ids = box[side].get('battingOrder', [])
                
                for slot, p_id in enumerate(order_ids, start=1):
                    player_info = box[side]['players'][f"ID{p_id}"]
                    
                    lineup_records.append({
                        "game_id": game_id,
                        "team": team_names[side],
                        "batting_order": slot,
                        "player_name": player_info['person']['fullName'],
                        "player_id": p_id,
                        "position": player_info['position']['abbreviation']
                    })
        except Exception as e:
            print(f"Lineup not yet available for {team_names['away']} @ {team_names['home']}")

    return pd.DataFrame(lineup_records)

# Test for a game day (using your June 15 example)
df_lineups = get_lineups_to_df('06/15/2025')
print(df_lineups.head(9)) # Show the lead-off through 9th hitter for the first game

In [ ]:
# Player's in the lineup that are due for a hit based on their luck score

# Assuming your Luck Score table is loaded as df_luck
df_today_luck = pd.merge(
    df_lineups, 
    luck_df[['player_id', 'luck_score', 'avg_exit_velo']], 
    on='player_id', 
    how='inner'
)

# Display the unluckiest players in today's lineup (due for a hit!)
due_for_hit = df_today_luck.sort_values('luck_score').head(5)

Get the standings by division

In [5]:
import statsapi
import pandas as pd

# Fetch raw standings data for both leagues
standings_raw = statsapi.standings_data(leagueId="103,104")

# Dictionary to store our DataFrames
division_dfs = {}

for div_id, div_data in standings_raw.items():
    div_name = div_data['div_name']
    teams_list = div_data['teams']
    
    # Convert the list of team dictionaries into a DataFrame
    df = pd.DataFrame(teams_list)
    
    # Clean up the DataFrame: keep only relevant columns
    cols_to_keep = ['name', 'w', 'l', 'pct', 'gb', 'wc_gb', 'streak']
    df = df[cols_to_keep]
    
    # Rename columns for better readability
    df.columns = ['Team', 'W', 'L', 'Win %', 'GB', 'WC GB', 'Streak']
    
    division_dfs[div_name] = df

# Example: Accessing a specific division
print("--- AL EAST ---")
print(division_dfs['American League East'])

KeyError: "['pct', 'streak'] not in index"

In [7]:
import statsapi

# Returns a dictionary structured by division
data = statsapi.standings_data(leagueId=103)

for division_id in data:
    division_name = data[division_id]['div_name']
    print(f"\n--- {division_name} ---")
    
    # Iterate through teams in the division
    for team in data[division_id]['teams']:
        name = team['name']
        w = team['w']
        l = team['l']
        gb = team['gb']
        print(f"{name}: {w}-{l} ({gb} GB)")


--- American League East ---
Toronto Blue Jays: 94-68 (- GB)
New York Yankees: 94-68 (- GB)
Boston Red Sox: 89-73 (5.0 GB)
Tampa Bay Rays: 77-85 (17.0 GB)
Baltimore Orioles: 75-87 (19.0 GB)

--- American League Central ---
Cleveland Guardians: 88-74 (- GB)
Detroit Tigers: 87-75 (1.0 GB)
Kansas City Royals: 82-80 (6.0 GB)
Minnesota Twins: 70-92 (18.0 GB)
Chicago White Sox: 60-102 (28.0 GB)

--- American League West ---
Seattle Mariners: 90-72 (- GB)
Houston Astros: 87-75 (3.0 GB)
Texas Rangers: 81-81 (9.0 GB)
Athletics: 76-86 (14.0 GB)
Los Angeles Angels: 72-90 (18.0 GB)


In [10]:
import statsapi

# Get raw data for one league
data = statsapi.standings_data(leagueId=103)

# Dig down to the first team in the first division found
first_division_key = list(data.keys())[0]
sample_team = data[first_division_key]['teams'][0]

# Print all available fields
print("Available Fields in 'teams' list:")
for field in sample_team.keys():
    print(f"- {field}")

Available Fields in 'teams' list:
- name
- div_rank
- w
- l
- gb
- wc_rank
- wc_gb
- wc_elim_num
- elim_num
- team_id
- league_rank
- sport_rank


This returns all the standings by division by League.
TODO need to add W%

In [ ]:
import statsapi
import pandas as pd

def get_division_standings_dfs():
    # leagueId 103 is AL, 104 is NL. Using both gets the whole league.
    standings_raw = statsapi.standings_data(leagueId="103,104")
    
    # This dictionary will hold { "Division Name": DataFrame }
    division_dict = {}

    for division_id in standings_raw:
        div_name = standings_raw[division_id]['div_name']
        teams_list = standings_raw[division_id]['teams']
        
        # Convert the list of team dicts directly to a DataFrame
        df = pd.DataFrame(teams_list)
        
        # Optional: Reorder or filter columns to keep it clean
        # TODO Calculate more columns if needed
        cols = ['name', 'w', 'l', 'gb', 'div_rank', 'wc_gb', 'wc_rank', 'league_rank']
        df = df[cols]
        
        # Set the Division Name as the key in the dictionary
        division_dict[div_name] = df
        
    return division_dict

# Execution
all_standings = get_division_standings_dfs()

# Example: Accessing just the NL West
print("--- National League West ---")
print(all_standings['National League West'])

--- National League West ---
                   name   w    l    gb div_rank wc_gb wc_rank league_rank
0   Los Angeles Dodgers  93   69     -        1     -       -           3
1      San Diego Padres  90   72   3.0        2  +7.0       2           5
2  San Francisco Giants  81   81  12.0        3   2.0       5           8
3  Arizona Diamondbacks  80   82  13.0        4   3.0       6           9
4      Colorado Rockies  43  119  50.0        5  40.0      12          15
